**POC for identifying groups of individuals who can live together at a new dwelling**


In [1]:
# imports
import os
import pandas as pd


In [2]:
# load in CSV of Persons
# expect single CSV to have all necessary info about a person
people_filepath = 'data/PeopleSheet.csv'
peeps = pd.read_csv(people_filepath)

In [3]:
# Create a Dwelling to test People against
# Note: no geo math, just flag whether, e.g., it is no_offenders (True)
dwelling_filepath = 'data/DwellingSheet.csv'
dwelling = pd.read_csv(dwelling_filepath)

In [10]:
# Test Persons against Dwelling, return list of eligible and eliminated
if not dwelling.offender_ok[0] == 'y':
    eligibles_loc = peeps[peeps['is_offender'] == 'n']  ## fragile: uppercase breaks
    eliminated_loc = peeps[peeps['is_offender'] == 'y']

print(f'# eligible: {len(eligibles_loc)}')
print(f'# eliminated: {len(eliminated_loc)}')

if len(eligibles_loc) + len(eliminated_loc) == len(peeps):
    print(f'All {len(peeps)} peeps accounted for!')    
else:
    print('Looks like we might have lost one! Started with X peeps.')
# test that all records went one place or the other (using .len?)

# eligible: 5
# eliminated: 2
All 7 peeps accounted for!


In [0]:
# Hard-group eligible persons based on a small number of lifestyle flags


In [0]:
# Report groups